# 9. Evaluate the Hybrid Restoration Pipeline

This notebook evaluates and compares the performance of three different image restoration methods:
1.  **Real-ESRGAN Only**: The baseline model for detail enhancement.
2.  **U-Net Only**: The PyTorch model trained for color and light correction.
3.  **Hybrid Pipeline (U-Net + Real-ESRGAN)**: The 2-step pipeline that first corrects color/light with the U-Net and then enhances details with Real-ESRGAN.

We will process a set of test images and compare the outputs both visually and quantitatively using PSNR and SSIM metrics against the ground truth.

### 1. Setup and Imports

Import all necessary libraries, define file paths, and set up the device (GPU/CPU).

In [2]:
import torch
import numpy as np
from PIL import Image
import os
import cv2
import pandas as pd
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt

# Make sure the project root is in the Python path
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..')))

from src.dl.hybrid_pipeline import HybridPipeline
from src.dl.realesrgan_wrapper import RealESRGANWrapper
from src.dl.pytorch_unet import UNet
from torchvision import transforms

# --- Configuration ---
UNET_MODEL_PATH = "../outputs/models/unet/best_unet_resnet34_perceptual.pth"
REALESRGAN_MODEL_STR = "x4"
TEST_IMAGE_DIR = "../data/raw/AI_for_Art_Restoration_2/paired_dataset_art/damaged"
GROUND_TRUTH_DIR = "../data/raw/AI_for_Art_Restoration_2/paired_dataset_art/undamaged"
OUTPUT_DIR = "../outputs/evaluation_hybrid"

# Select a few images for testing
TEST_IMAGES = ["1.png", "2.png", "3.png", "4.jpg", "5.jpg"]

# --- Setup ---
os.makedirs(OUTPUT_DIR, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Check if U-Net model exists
if not os.path.exists(UNET_MODEL_PATH):
    print(f"ERROR: U-Net model not found at {UNET_MODEL_PATH}")
    print("Please train the U-Net model first by running 'src/training/train_unet.py'")
else:
    print("U-Net model found.")


d:\R&D Project\image_processing\venv\Lib\site-packages\torchvision\transforms\functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


Using device: cuda
U-Net model found.


### 2. Load Models

Instantiate all three models:
- The standalone `RealESRGANWrapper`.
- The standalone `UNet`.
- The combined `HybridPipeline`.

In [3]:
%pip install segmentation-models-pytorch
import segmentation_models_pytorch as smp

# --- Load U-Net Model ---
# The model was trained using segmentation-models-pytorch with a resnet34 backbone
unet_model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights=None, # Not loading pretrained imagenet weights, will load our own
    in_channels=3,
    classes=3,
)
if os.path.exists(UNET_MODEL_PATH):
    # The saved state dict is from the model itself, not a checkpoint dictionary
    state_dict = torch.load(UNET_MODEL_PATH, map_location=device)
    unet_model.load_state_dict(state_dict)
    unet_model.to(device)
    unet_model.eval()
    print("U-Net model loaded successfully.")

# --- Load Real-ESRGAN Model ---
realesrgan_model = RealESRGANWrapper(model_str=REALESRGAN_MODEL_STR)
print("Real-ESRGAN model loaded successfully.")

# --- Load Hybrid Pipeline ---
hybrid_pipeline = None
if os.path.exists(UNET_MODEL_PATH):
    hybrid_pipeline = HybridPipeline(UNET_MODEL_PATH, REALESRGAN_MODEL_STR, device)
    print("Hybrid pipeline loaded successfully.")

# Transformation for U-Net input
unet_transform = transforms.Compose([
    transforms.ToTensor(),
])

Note: you may need to restart the kernel to use updated packages.
U-Net model loaded successfully.
Loaded Real-ESRGAN: RealESRGAN_x4plus | scale=4 | device=cuda
Real-ESRGAN model loaded successfully.
HybridPipeline using device: cuda
Loading U-Net model from ../outputs/models/unet/best_unet_resnet34_perceptual.pth...
U-Net model loaded successfully.
Loading Real-ESRGAN model 'x4'...
Loaded Real-ESRGAN: RealESRGAN_x4plus | scale=4 | device=cuda
Real-ESRGAN model loaded successfully.
Loading GFPGAN for face enhancement...


d:\R&D Project\image_processing\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\R&D Project\image_processing\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


GFPGAN model loaded successfully.
Hybrid pipeline loaded successfully.


### 3. Define Helper and Metric Functions

Create helper functions to:
- Run each restoration model on an image.
- Calculate PSNR and SSIM between two images.
- Display the results in a grid.

In [4]:
def restore_realesrgan_only(img_bgr):
    """Restores an image using only Real-ESRGAN."""
    # img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    restored_img_bgr, _ = realesrgan_model.restore(img_bgr)
    print(f" Type of restored image under restore_realesrgan_only() is {type(restored_img_bgr)}")
    print(restored_img_bgr)
    restored_img_bgr = restored_img_bgr[0]
    return cv2.cvtColor(restored_img_bgr, cv2.COLOR_BGR2RGB)

def restore_unet_only(img_pil):
    """Restores an image using only the U-Net model."""
    if not os.path.exists(UNET_MODEL_PATH): return None
    
    # img = Image.open(image_path).convert("RGB")
    with torch.no_grad():
        input_tensor = unet_transform(img_pil).unsqueeze(0).to(device)
        output_tensor = unet_model(input_tensor)
    
    # Convert tensor to PIL Image
    output_img = transforms.ToPILImage()(output_tensor.squeeze(0).cpu())
    return np.array(output_img)

def restore_hybrid(img_pil):
    """Restores an image using the full hybrid pipeline."""
    if hybrid_pipeline is None: return None

    # The hybrid pipeline's restore_image method saves to a file, 
    # so we'll adapt it to return the image instead for this notebook.
    # img = Image.open(image_path).convert("RGB")
    
    # Step 1: U-Net
    with torch.no_grad():
        input_tensor = unet_transform(img_pil).unsqueeze(0).to(device)
        unet_output_tensor = hybrid_pipeline.unet(input_tensor)
        unet_output_bgr = hybrid_pipeline.tensor_to_cv2(unet_output_tensor)
        
    # Step 2: Real-ESRGAN
    # unet_output_img_bgr = cv2.cvtColor(unet_output_img_rgb, cv2.COLOR_RGB2BGR)
    final_output_bgr, _ = hybrid_pipeline.realesrgan.restore(unet_output_bgr)
    return cv2.cvtColor(final_output_bgr, cv2.COLOR_BGR2RGB)

def calculate_metrics(img1, img2):
    """Calculates PSNR and SSIM between two images."""
    img1_np = np.array(img1)
    img2_np = np.array(img2)
    
    # Ensure images are the same size
    h, w, _ = img1_np.shape
    img2_np = cv2.resize(img2_np, (w, h))
    
    psnr_val = psnr(img1_np, img2_np, data_range=255)
    ssim_val = ssim(img1_np, img2_np, channel_axis=2, data_range=255)
    return psnr_val, ssim_val

def plot_results(images, titles, figsize=(20, 10)):
    """Plots a list of images with their titles."""
    plt.figure(figsize=figsize)
    for i, (img, title) in enumerate(zip(images, titles)):
        plt.subplot(1, len(images), i + 1)
        plt.imshow(img)
        plt.title(title)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

### 4. Run Evaluation Loop

Iterate through the test images, apply each restoration method, calculate metrics, and display the results.

In [5]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_summary())
print(torch.cuda.empty_cache())
print(torch.cuda.memory_allocated())

986323456
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    940 MiB |    940 MiB |   1035 MiB |  96785 KiB |
|       from large pool |    729 MiB |    729 MiB |    817 MiB |  89344 KiB |
|       from small pool |    210 MiB |    210 MiB |    218 MiB |   7441 KiB |
|---------------------------------------------------------------------------|
| Active memory         |    940 MiB |    940 MiB |   1035 MiB |  96785 KiB |
|       from large pool |    729 MiB |    729 MiB |   

In [6]:
results_data = []

for img_name in TEST_IMAGES:
    print(f"--- Processing: {img_name} ---")
    torch.cuda.empty_cache()
    damaged_path = os.path.join(TEST_IMAGE_DIR, img_name)
    truth_path = os.path.join(GROUND_TRUTH_DIR, img_name)

    if not os.path.exists(damaged_path) or not os.path.exists(truth_path):
        print(f"Skipping {img_name}, file not found.")
        continue

    # Load images
    damaged_img_pil = Image.open(damaged_path).convert("RGB")
    print(type(damaged_img_pil))
    ground_truth_img = Image.open(truth_path).convert("RGB")
    print(type(ground_truth_img))

    # --- Run Restoration ---
    damaged_img_np = np.array(damaged_img_pil)
    damaged_image_bgr = cv2.cvtColor(damaged_img_np, cv2.COLOR_RGB2BGR)
    
    realesrgan_restored = restore_realesrgan_only(damaged_image_bgr)
    unet_restored = restore_unet_only(damaged_img_pil)
    hybrid_restored = restore_hybrid(damaged_img_pil)

    if unet_restored is None or hybrid_restored is None:
        print("Skipping evaluation because U-Net model is not trained.")
        break

    # --- Calculate Metrics ---
    psnr_damaged, ssim_damaged = calculate_metrics(ground_truth_img, damaged_img_pil)
    psnr_realesrgan, ssim_realesrgan = calculate_metrics(ground_truth_img, realesrgan_restored)
    psnr_unet, ssim_unet = calculate_metrics(ground_truth_img, unet_restored)
    psnr_hybrid, ssim_hybrid = calculate_metrics(ground_truth_img, hybrid_restored)

    # --- Store Results ---
    results_data.append({
        "Image": img_name,
        "PSNR_Damaged": psnr_damaged, "SSIM_Damaged": ssim_damaged,
        "PSNR_U-Net": psnr_unet, "SSIM_U-Net": ssim_unet,
        "PSNR_RealESRGAN": psnr_realesrgan, "SSIM_RealESRGAN": ssim_realesrgan,
        "PSNR_Hybrid": psnr_hybrid, "SSIM_Hybrid": ssim_hybrid,
    })

    # --- Display Visuals ---
    images_to_plot = [damaged_img_pil, ground_truth_img, unet_restored, realesrgan_restored, hybrid_restored]
    titles = [
        f"Damaged\nPSNR: {psnr_damaged:.2f}, SSIM: {ssim_damaged:.3f}",
        "Ground Truth",
        f"U-Net Only\nPSNR: {psnr_unet:.2f}, SSIM: {ssim_unet:.3f}",
        f"Real-ESRGAN Only\nPSNR: {psnr_realesrgan:.2f}, SSIM: {ssim_realesrgan:.3f}",
        f"Hybrid\nPSNR: {psnr_hybrid:.2f}, SSIM: {ssim_hybrid:.3f}"
    ]
    plot_results(images_to_plot, titles)

# --- Create and display summary DataFrame ---
if results_data:
    df_results = pd.DataFrame(results_data)
    display(df_results)
else:
    print("\nNo results to display. Please ensure the U-Net model is trained and paths are correct.")


--- Processing: 1.png ---
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### 5. Conclusion

The table and images above provide a comprehensive comparison.

- **Damaged**: The baseline scores for the input images.
- **U-Net Only**: Should show significant improvement in color and lighting, which will be reflected in a higher PSNR/SSIM compared to the damaged version. The image might still appear blurry.
- **Real-ESRGAN Only**: Should produce a sharp image, but the colors might be incorrect or washed out, similar to the damaged input. The PSNR/SSIM might not be high if the colors are wrong.
- **Hybrid**: This should ideally have the best of both worlds: corrected colors from the U-Net and enhanced details from Real-ESRGAN, leading to the highest PSNR and SSIM scores.

Based on the results, we can determine the effectiveness of the hybrid approach.